<a href="https://colab.research.google.com/github/Eminent01/AMMI_Research/blob/main/AMMI_Research2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import sys
# import os
# # import config
# import pandas as pd
# import numpy as np
# # from config import config
# import matplotlib.pylab as plt

# import yfinance as yf
# from pandas_datareader import data as pdr
# import pkg_resources
# %matplotlib inline

In [2]:
# from pandas_datareader import data
# import matplotlib.pyplot as plt
# import pandas as pd

In [3]:
# # Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
# tickers = ['AAPL', 'MSFT', '^GSPC']

# # We would like all available data from 01/01/2000 until 12/31/2016.
# start_date = '2010-01-01'
# end_date = '2016-12-31'

# # User pandas_reader.data.DataReader to load the desired data. As simple as that.
# panel_data = data.DataReader('INPX', 'google', start_date, end_date)

In [4]:
# start_date = '2016-01-01'
# end_date = '2020-12-31'

In [5]:
# panel_data = data.DataReader(ticker_list, start_date, end_date)

In [6]:
# panel_data.to_frame().head(9)

In [7]:
# df = pdr.get_data_yahoo([ticker_list][0], 
#                           start='2008-01-01', end="2021-01-01")

In [8]:
# data = df.copy()

In [9]:
# data = data.stack().reset_index()
# data.columns.names = [None]
# data = data.drop(['Close'], axis=1)
# data.head()

### Download data from Yfinance API

In [10]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 630 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=312b087d1a6005889b20e0511621704d869eebcb06f2da47c7472129a11f8176
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [11]:
ticker_list = ['WBA', 'TRV', 'MMM', 'RTX', 'DD', 'KO', 'V', 'XOM', 'CAT', 'PG',
       'AXP', 'WMT', 'MSFT', 'VZ', 'JPM', 'UNH', 'DIS', 'HD', 'AAPL', 'CVX',
       'JNJ', 'CSCO', 'PFE', 'IBM', 'MRK', 'INTC', 'MCD', 'BA', 'NKE', 'GS']

In [12]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

df = yf.download(ticker_list, 
                      start='2010-01-01', 
                      end='2021-06-12', 
                      progress=False,
)
print(df.head())

           Adj Close                                                         \
                AAPL        AXP         BA        CAT       CSCO        CVX   
Date                                                                          
2010-01-04  6.515211  33.792698  43.777550  41.353176  17.533108  47.266056   
2010-01-05  6.526478  33.718361  45.211349  41.847603  17.454987  47.600853   
2010-01-06  6.422664  34.263416  46.582806  41.974712  17.341370  47.606831   
2010-01-07  6.410792  34.819126  48.468563  42.144218  17.419487  47.427486   
2010-01-08  6.453412  34.794250  48.001015  42.617458  17.511803  47.511177   

                                                         ...    Volume  \
                   DD        DIS          GS         HD  ...       PFE   
Date                                                     ...             
2010-01-04  29.345999  27.933924  141.788162  21.250742  ...  54898644   
2010-01-05  30.653408  27.864237  144.294876  21.406395  ...  45714931 

In [13]:
data = df.copy()

In [14]:
data = data.stack().reset_index()
data.columns.names = [None]
data = data.drop(['Close'], axis=1)

In [15]:
data.head()

,Date,level_1,Adj Close,High,Low,Open,Volume
0,2010-01-04,AAPL,6.515211,7.660714,7.585000,7.622500,493729600
1,2010-01-04,AXP,33.792698,41.099998,40.389999,40.810001,6894300
2,2010-01-04,BA,43.777550,56.389999,54.799999,55.720001,6186700
3,2010-01-04,CAT,41.353176,59.189999,57.509998,57.650002,7325600
4,2010-01-04,CSCO,17.533108,24.840000,24.010000,24.110001,59853700


In [16]:
data.columns = ['date','stock_name','close','high','low','open','volume']

In [17]:
data.head()

,date,stock_name,close,high,low,open,volume
0,2010-01-04,AAPL,6.515211,7.660714,7.585000,7.622500,493729600
1,2010-01-04,AXP,33.792698,41.099998,40.389999,40.810001,6894300
2,2010-01-04,BA,43.777550,56.389999,54.799999,55.720001,6186700
3,2010-01-04,CAT,41.353176,59.189999,57.509998,57.650002,7325600
4,2010-01-04,CSCO,17.533108,24.840000,24.010000,24.110001,59853700


In [18]:
# check for missing values
data.isna().sum()

date          0
stock_name    0
close         0
high          0
low           0
open          0
volume        0
dtype: int64

In [19]:
# check if all stock have the same number of data points

no_datasets = []
for i in ticker_list:
    no_data_points = data[data['stock_name']==i].shape[0]
    no_datasets.append((i,no_data_points))
    data_points_df = pd.DataFrame(no_datasets)

In [20]:
data_points_df.head()

,0,1
0,WBA,2880
1,TRV,2880
2,MMM,2880
3,RTX,2880
4,DD,2880


In [21]:
##Save the data to CSV
data.to_csv('datasets.csv', index=False)

#### Create close price data



In [22]:
# read the data from the saved csv file
df_prices = pd.read_csv('/content/datasets.csv')

In [23]:
# Reset the Index to tic and date
df_prices = df_prices.reset_index().set_index(['stock_name', 'date']).sort_index()

# Get the list of all the tickers
tic_list = list(set([i for i,j in df_prices.index]))

# Create an empty data frame for the close prices
df_close = pd.DataFrame()

In [24]:
# Reset the Index to tic and date
df_prices = df_prices.reset_index().set_index(['stock_name', 'date']).sort_index()

# Get all the Close Prices
df_close = pd.DataFrame()

for ticker in tic_list:
    series = df_prices.xs(ticker).close
    df_close[ticker] = series
    
df_close = df_close.reset_index()

In [25]:
df_close.head()

,date,PFE,DIS,PG,JPM,BA,RTX,TRV,DD,CAT,...,AAPL,GS,JNJ,WBA,INTC,HD,CVX,XOM,MMM,KO
0,2010-01-04,11.128050,27.933924,41.439651,30.742662,43.777550,33.066311,36.491493,29.345999,41.353176,...,6.515211,141.788162,44.278530,26.326166,14.020260,21.250742,47.266056,41.998314,57.456116,19.216944
1,2010-01-05,10.969330,27.864237,41.453205,31.338144,45.211349,32.572353,35.627010,30.653408,41.847603,...,6.526478,144.294876,43.765099,26.114424,14.013545,21.406395,47.600853,42.162292,57.096218,18.984482
2,2010-01-06,10.934063,27.716160,41.256580,31.510344,46.582806,32.401577,35.121494,31.196468,41.974712,...,6.422664,142.754761,44.121075,25.916800,13.966546,21.332270,47.606831,42.526711,57.905972,18.977741
3,2010-01-07,10.892910,27.724876,41.032841,32.134518,48.468563,32.540058,35.627010,31.216583,42.144218,...,6.410792,145.548309,43.806164,26.072081,13.832252,21.584295,47.427486,42.393082,57.947495,18.930578
4,2010-01-08,10.981089,27.768425,40.978596,32.055595,48.001015,32.604687,35.575722,31.327204,42.617458,...,6.453412,142.795715,43.956768,26.107367,13.986689,21.480520,47.511177,42.223019,58.355804,18.580204


In [26]:
# Save the Close Price datase
df_close.to_csv('close_prices.csv', index=False)

In [27]:
# ! pip install pandas-ta

In [28]:
# import matplotlib.pyplot as plt

In [29]:
# plt.plot(df)
# plt.xlabel("Date")
# plt.ylabel("Stock Price")

## Stock Selection using the stocks with least reconstruction error stock usind Auto encoder network

In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import regularizers, optimizers

from sklearn import preprocessing

In [30]:
close_price =pd.read_csv("/content/close_prices.csv")

In [31]:
df2 = close_price.copy()

In [32]:
df2 = df2.reset_index(drop=True).set_index(['date'])

In [33]:
df2.head()

,PFE,DIS,PG,JPM,BA,RTX,TRV,DD,CAT,MSFT,...,AAPL,GS,JNJ,WBA,INTC,HD,CVX,XOM,MMM,KO
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,11.128050,27.933924,41.439651,30.742662,43.777550,33.066311,36.491493,29.345999,41.353176,23.683075,...,6.515211,141.788162,44.278530,26.326166,14.020260,21.250742,47.266056,41.998314,57.456116,19.216944
2010-01-05,10.969330,27.864237,41.453205,31.338144,45.211349,32.572353,35.627010,30.653408,41.847603,23.690723,...,6.526478,144.294876,43.765099,26.114424,14.013545,21.406395,47.600853,42.162292,57.096218,18.984482
2010-01-06,10.934063,27.716160,41.256580,31.510344,46.582806,32.401577,35.121494,31.196468,41.974712,23.545334,...,6.422664,142.754761,44.121075,25.916800,13.966546,21.332270,47.606831,42.526711,57.905972,18.977741
2010-01-07,10.892910,27.724876,41.032841,32.134518,48.468563,32.540058,35.627010,31.216583,42.144218,23.300467,...,6.410792,145.548309,43.806164,26.072081,13.832252,21.584295,47.427486,42.393082,57.947495,18.930578
2010-01-08,10.981089,27.768425,40.978596,32.055595,48.001015,32.604687,35.575722,31.327204,42.617458,23.461166,...,6.453412,142.795715,43.956768,26.107367,13.986689,21.480520,47.511177,42.223019,58.355804,18.580204


In [36]:
def defineAutoencoder(num_stock, encoding_dim = 5, verbose=0):
    
    """
    Function for fitting an Autoencoder
    """

    # connect all layers
    input = Input(shape=(num_stock,))

    encoded = Dense(encoding_dim, kernel_regularizer=regularizers.l2(0.00001),name ='Encoder_Input')(input)

    decoded = Dense(num_stock, kernel_regularizer=regularizers.l2(0.00001), name ='Decoder_Input')(encoded)
    decoded = Activation("linear", name='Decoder_Activation_function')(decoded)

    # construct and compile AE model
    autoencoder = Model(inputs=input, outputs=decoded)
    adam = optimizers.Adam(lr=0.0005)
    autoencoder.compile(optimizer=adam, loss='mean_squared_error')
    if verbose!= 0:
        autoencoder.summary()

    return autoencoder

In [37]:
def getReconstructionErrorsDF(df_pct_change, reconstructed_data):
    
    """
    Function for calculating the reconstruction Errors
    """
    array = []
    stocks_ranked = []
    num_columns = reconstructed_data.shape[1]
    for i in range(0, num_columns):
        diff = np.linalg.norm((df_pct_change.iloc[:, i] - reconstructed_data[:, i]))  # 2 norm difference
        array.append(float(diff))

    ranking = np.array(array).argsort()
    r = 1
    for stock_index in ranking:
        stocks_ranked.append([ r
                              ,stock_index
                              ,df_pct_change.iloc[:, stock_index].name
                              ,array[stock_index]
                              ])
        r = r + 1

    columns = ['ranking','stock_index', 'stock_name' ,'recreation_error']
    df = pd.DataFrame(stocks_ranked, columns=columns)
    df = df.set_index('stock_name')
    return df

In [ ]:
df_pct_change = df.pct_change(1).astype(float)
df_pct_change = df_pct_change.replace([np.inf, -np.inf], np.nan)
df_pct_change = df_pct_change.fillna(method='ffill')

# the percentage change function will make the first two rows equal to nan
df_pct_change = df_pct_change.tail(len(df_pct_change) - 2)

In [ ]:
# remove columns where there is no change over a longer time period
df_pct_change = df_pct_change[df_pct_change.columns[((df_pct_change == 0).mean() <= 0.05)]]

In [ ]:
df_pct_change.head()

In [ ]:
# define the input parameters
hidden_layers = 5
batch_size = 500
epochs = 500
stock_selection_number = 20
num_stock = df_pct_change.shape[1]
verbose = 1

In [ ]:
# Normalize the data
df_scaler = preprocessing.MinMaxScaler()
df_pct_change_normalised = df_scaler.fit_transform(df_pct_change)

In [ ]:
# define autoencoder
print('-' * 25 + 'Define autoencoder model')
num_stock = len(df_pct_change.columns)
autoencoder = defineAutoencoder(num_stock=num_stock, encoding_dim=hidden_layers, verbose=verbose)
#plot_model(autoencoder, to_file='img/model_autoencoder_1.png', show_shapes=True,
#           show_layer_names=True)

In [ ]:
# train autoencoder
print('-' * 25 + 'Train autoencoder model')
autoencoder.fit(df_pct_change_normalised, df_pct_change_normalised, shuffle=False, epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)